In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Tham khảo: https://gist.github.com/lttn1204/0ee0d5fc05729876d403bd5d48ca5d7d

##Install lib

In [ ]:
!pip install pyvex
!pip install matplotlib-venn
!pip install archinfo
!pip install bitstring
!pip install angr
!pip install analyse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.2/156.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.0/195.0 kB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pyvex
import archinfo
import re
import hashlib
import os
import random
import pickle

In [ ]:
# this function to get all variable of one statement and it's type (define or reference)
def get_var_in_stmt(stmt):
    res=[]
    # regular expression pattern to match temporary variables
    pattern = r't\d+'
    variables = re.findall(pattern, stmt)
    if stmt[:2]=='if':
        res.append([variables[0],'ref'])
        return res
    if len(variables)==0:
        return []
    eql=stmt.index('=')
    for var in variables:
        if stmt.index(var)<eql:
            res.append([var,'def'])
        else:
            res.append([var,'ref'])
    return res

# this function detect a statement ca be propagation or not
def dectect_which_statement_can_be_propagation(stmt):
    vars=get_var_in_stmt(stmt)
    if len(stmt)<10 and len(vars)==2 and vars[0][1]=='def' and vars[1][1]=='ref':
        return True
    return False

In [ ]:
# implement dead code elimination in a block
def dead_code_elimination(block):
    res=[]
    for i in range(len(block)):
        if dectect_which_statement_can_be_propagation(block[i])==False:
            res.append(block[i])
    return res
# this function return reference variable in a statement in basic block
def reference(stmt):
    res=[]
    vars=get_var_in_stmt(stmt)
    for var in vars:
        if var[1]=='ref':
            res.append(var[0])
    return res

# this function return define variable in a statement in basic block
def define(stmt):
    res=[]
    vars=get_var_in_stmt(stmt)
    for var in vars:
        if var[1]=='def':
            res.append(var[0])
    return res

# this function return intersection of 2 list of variables
def intersect(vars1,vars2):
    for var in vars1:
        if var in vars2:
            return True
    return False


# this function return union of 2 list of variables
def union(vars1,vars2):
    res=[]
    for var in vars1:
        res.append(var)
    for var in vars2:
        res.append(var)
    return res

In [ ]:
def parse_bb_to_strand(block):
    S=[]
    uncovered=list(range(len(block)))
    #print(uncovered)
    while len(uncovered)>0:
        last=uncovered.pop()
        strand=[block[last]]
        used=reference(block[last])
        for i in range(last-1,-1,-1):
            #print(f" i: {i}")
            #print(f"block:  {block[i]}")
            needed=intersect(used,define(block[i]))
            #print(f"define: {define(block[i])}")
            if needed==True:
                #print("found")
                strand.append(block[i])
                used= union(reference(block[i]),used)
                #print(f"used: {used}")
                #print(f" i: {uncovered}")
                try:
                    uncovered.remove(i)
                except:
                    continue
                #print(f" i: {uncovered}")
        S.append(strand)
    return S



In [ ]:
def copy_propagation(block):
    ACP={}
    res=[]
    for i in range(len(block)):
        #print("block: ", block[i])
        stmt=block[i]
        vars=get_var_in_stmt(stmt)[::-1]
        #print("vars:", vars)
        for var in vars:
            #print(var)
            if var[1]=='ref' and var[0] in ACP.keys():
                #print("replace", var[0],ACP[var[0]])
                stmt=stmt.replace(var[0],ACP[var[0]])
                #print("stmt: ", stmt)
            if var[1]=='def' and var[0] in ACP.keys():
                #print("dell",ACP[var[0]])
                del ACP[var[0]]
        if dectect_which_statement_can_be_propagation(stmt)==True:
            vars=get_var_in_stmt(stmt)
            ACP[vars[0][0]]=vars[1][0]
            #print("ACP:",ACP)
        res.append(stmt)
        #print("----------------------")
    return res

In [ ]:
# implement dead code elimination in a block
def dead_code_elimination(block):
    res=[]
    for i in range(len(block)):
        if dectect_which_statement_can_be_propagation(block[i])==False:
            res.append(block[i])
    return res

In [ ]:
# this funtion normalize a basic block, rename all temporary variables in descending chronological order
def normalize(block):

    mapping={}
    pattern = r't\d+'
    all_stmt="\n".join(block)
    if get_var_in_stmt(all_stmt)==[]:
        return block
    print(all_stmt)
    variables = re.findall(pattern, all_stmt)
    variables = list(dict.fromkeys(variables))

    for i in range(len(variables)):
        mapping[variables[i]]='t'+str(i)
    print(mapping)
    def new_replace(match):
        return mapping[match.group(0)]

    pattern = re.compile('|'.join(map(re.escape, mapping.keys())))
    result = pattern.sub(new_replace, all_stmt)
    print("---------------------")

    print("---------------------")
    print(result)
    return result

In [ ]:
# this function return vector from a list of strand
def extract_vector_from_strand(liststrand, size):
    vector=[0]*size
    print("Hash values:\n")
    for strand in liststrand:
        value=int(hashlib.md5(str(strand).encode()).hexdigest(),16)%2**10
        print(value)
        vector[value]+=1
    return vector

In [ ]:
def get_vexIR(raw_bytes):
    irsb = pyvex.IRSB(data=raw_bytes, mem_addr=0x08048429, arch=archinfo.ArchX86())
    result=b''
    for stmt in irsb.statements:
        if not isinstance(stmt, pyvex.IRStmt.IMark):
            result+=(str(stmt).encode()+b'\n')
    return result.decode()

def process_a_func(raw_bytes,size):
    vex_ir=get_vexIR(raw_bytes)

    block=vex_ir.split("\n")[:-1]

    strand=parse_bb_to_strand(block)

    res=[]
    for i in strand:
        res.append(copy_propagation(i))
    strand=res

    res=[]
    for i in strand:
        res.append(dead_code_elimination(i))
    strand=res

    res=[]
    for i in strand:
        res.append(normalize(i))
    strand=res

    vector=extract_vector_from_strand(strand,size)

    print("Vector:\n ",vector)

In [ ]:
data = bytes.fromhex("4d 5a 90000300000004000000ffff0000b800000000000000400000000000000000000000000000000000000000000000000000000000000000000000f80000000e1fba0e00b409cd21b8014ccd21546869732070726f6772616d2063616e6e6f742062652072756e20696e20444f53206d6f64652e0d0d0a24000000000000003876adf97c17c3aa7c17c3aa7c17c3aa756f50aa7a17c3aa7c17c2aa5417c3aa376fc2ab7517c3aa376fc7ab6e17c3aa376fc0ab7f17c3aa376fcbab7f17c3aa376fc6ab7e17c3aa376f3caa7d17c3aa376fc1ab7d17c3aa526963687c17c3aa00000000000000000000")
vector_size = 1000
process_a_func(data, vector_size)

PUT(offset=48) = t39
t39 = 8Uto32(t4)
t4 = GET:I8(offset=8)
{'t39': 't0', 't4': 't1'}
---------------------
---------------------
PUT(offset=48) = t0
t0 = 8Uto32(t1)
t1 = GET:I8(offset=8)
PUT(offset=44) = t38
t38 = 8Uto32(t17)
t17 = LDle:I8(t10)
STle(t10) = t7
t7 = Add8(t9,t4)
t9 = LDle:I8(t10)
t10 = GET:I32(offset=8)
t4 = GET:I8(offset=8)
{'t38': 't0', 't17': 't1', 't10': 't2', 't7': 't3', 't9': 't4', 't4': 't5'}
---------------------
---------------------
PUT(offset=44) = t0
t0 = 8Uto32(t1)
t1 = LDle:I8(t2)
STle(t2) = t3
t3 = Add8(t4,t5)
t4 = LDle:I8(t2)
t2 = GET:I32(offset=8)
t5 = GET:I8(offset=8)
STle(t10) = t15
t15 = Add8(t17,t4)
t17 = LDle:I8(t10)
STle(t10) = t7
t7 = Add8(t9,t4)
t9 = LDle:I8(t10)
t10 = GET:I32(offset=8)
t4 = GET:I8(offset=8)
{'t10': 't0', 't15': 't1', 't17': 't2', 't4': 't3', 't7': 't4', 't9': 't5'}
---------------------
---------------------
STle(t0) = t1
t1 = Add8(t2,t3)
t2 = LDle:I8(t0)
STle(t0) = t4
t4 = Add8(t5,t3)
t5 = LDle:I8(t0)
t0 = GET:I32(offset=8)
t3 

In [ ]:
with open('/content/gdrive/MyDrive/calc_rawbyte.txt', 'r') as file:
    hex_string = file.read().strip()
print(hex_string)
data = bytes.fromhex(hex_string)
print(data)
vector_size = 1000
process_a_func(data, vector_size)

4d 5a 90 00 03 00 00 00 04 00 00 00 ff ff 00 00 b8 00 00 00 00 00 00 00 40 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 f8 00 00 00 0e 1f ba 0e 00 b4 09 cd 21 b8 01 4c cd 21 54 68 69 73 20 70 72 6f 67 72 61 6d 20 63 61 6e 6e 6f 74 20 62 65 20 72 75 6e 20 69 6e 20 44 4f 53 20 6d 6f 64 65 2e 0d 0d 0a 24 00 00 00 00 00 00 00 38 76 ad f9 7c 17 c3 aa 7c 17 c3 aa 7c 17 c3 aa 75 6f 50 aa 7a 17 c3 aa 7c 17 c2 aa 54 17 c3 aa 37 6f c2 ab 75 17 c3 aa 37 6f c7 ab 6e 17 c3 aa 37 6f c0 ab 7f 17 c3 aa 37 6f cb ab 7f 17 c3 aa 37 6f c6 ab 7e 17 c3 aa 37 6f 3c aa 7d 17 c3 aa 37 6f c1 ab 7d 17 c3 aa 52 69 63 68 7c 17 c3 aa 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 50 45 00 00 64 86 06 00 3a 45 a1 7b 00 00 00 00 00 00 00 00 f0 00 22 00 0b 02 0e 1e 00 10 00 00 00 90 00 00 00 00 00 00 50 17 00 00 00 10 00 00 00 00 00 40 01 00 00 00 00 10 00 00 00 10 00 00 0a 00 00 00 0a 00 00 00 0a 00 00 00 00 00 00 00 00 b0 00 00 00 1